In [1]:
import os
import csv
import shutil
import pathlib
from ultralytics import YOLO
from PIL import Image

In [4]:
dataset_dir = "/Users/jayden/Library/CloudStorage/OneDrive-jaydentang/Datasets/FallDetectionDataset-2017IAPR"
dataset_yolo_dir = "/Users/jayden/Library/CloudStorage/OneDrive-jaydentang/Datasets/FallDetectionDataset-2017IAPR-yolo"
model = YOLO("./yolov8n_person.pt")

dirs1 = ["images", "labels"]
dirs2 = ["train", "val", "test"]

for dir1 in dirs1:
    for dir2 in dirs2:
        pathlib.Path(os.path.join(dataset_yolo_dir, dir1, dir2)).mkdir(parents=True, exist_ok=True)

error = [[0, 0]]

for f1 in os.listdir(dataset_dir):
    if f1 == "train" or f1 == "val":
        print(f1)
        for f2 in os.listdir(f"{dataset_dir}/{f1}"):
            if os.path.isdir(f"{dataset_dir}/{f1}/{f2}"):
                with open(f"{dataset_dir}/{f1}/{f2}/labels.csv") as f:
                    labels = list(csv.reader(f))
                for f3 in os.listdir(f"{dataset_dir}/{f1}/{f2}/rgb"):
                    if f3.endswith("png"):
                        results = model(f"{dataset_dir}/{f1}/{f2}/rgb/{f3}")
                        boxes = results[0].boxes.xyxy.cpu()
                        clss = results[0].boxes.cls.cpu().tolist()
                        box0 = []
                        
                        error[0][0] += 1
                        if 0.0 not in clss or len(clss) != len(set(clss)):
                            error[0][1] += 1
                            error.append([f"{f2}_{f3}", clss])
                            for r in results:
                                im_array = r.plot()
                                im = Image.fromarray(im_array[..., ::-1])
                                im.show()
                            while True:
                                if input() == "q":
                                    break

                        for box, cls in zip(boxes, clss):
                            if cls == 0:
                                box_list = box.tolist()
                                box_list.insert(0, labels[int(f3[4:-4])][1])
                                box0.append(box_list)
                        shutil.copyfile(f"{dataset_dir}/{f1}/{f2}/rgb/{f3}", f"{dataset_yolo_dir}/images/{f1}/{f2}_{f3}")
                        with open(f"{dataset_yolo_dir}/labels/{f1}/{f2}_{f3[:-4]}.txt", "w") as f:
                            for e1 in box0:
                               f.write(" ".join(map(str, e1)) + "\n")
                print(error)
                print("1")



val


FileNotFoundError: [Errno 2] No such file or directory: '/Users/jayden/Library/CloudStorage/OneDrive-jaydentang/Datasets/FallDetectionDataset-2017IAPR/val'